# Sapeceship

In [1]:
import pandas as pd
train_data = pd.read_csv('train_space.csv')
test_X = pd.read_csv('test_space.csv')
test_y = pd.read_csv('sample_submission_space.csv')

# Data Exploration

In [2]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
test_X.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [4]:
test_y.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

# Pre-Process

In [97]:
encoded_train = train_data.copy()
encoded_test_X = test_X.copy()
encoded_test_y = test_y.copy()

In [98]:
def drop_col(df,col):
    for i in col:
        df.drop(i,axis=1,inplace = True)

In [99]:
col = ['PassengerId','CryoSleep','Cabin','VIP','Name']
drop_col(encoded_train,col)
drop_col(encoded_test_X,col)

In [100]:
drop_col(encoded_test_y,['PassengerId'])

In [101]:
def fill_null(df,col):
    for i in col:
        q1 = df[i].quantile(0.25)
        q3 = df[i].quantile(0.75)
        iqr = q3 - q1
        lower_lim = q1 - 1.5*iqr
        upper_lim = q3 + 1.5*iqr
        without_outlier = df[i].apply(lambda x : None if x < lower_lim or x > upper_lim else x)
        without_outlier.fillna(without_outlier.mean(),inplace = True)
        df[i] = without_outlier
def fill_obj(df,col):
    for i in col:
        df[i].fillna(df[i].mode()[0],inplace = True)

In [102]:
def not_obj(df):
    col = []
    for i in df.columns:
        if df[i].dtype != 'O' and df[i].dtype != 'bool':
            col.append(i)
    return col
def obj(df):
    col = []
    for i in df.columns:
        if df[i].dtype == 'O':
            col.append(i)
    return col

In [103]:
col = not_obj(encoded_train)
fill_null(encoded_train,col)

In [104]:
col = not_obj(encoded_test_X)
fill_null(encoded_test_X,col)

In [105]:
col = obj(encoded_train)
fill_obj(encoded_train,col)

C:\Users\syedb\AppData\Local\Temp\ipykernel_17160\932789160.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(df[i].mode()[0],inplace = True)


In [106]:
col = obj(encoded_test_X)
fill_obj(encoded_test_X,col)

C:\Users\syedb\AppData\Local\Temp\ipykernel_17160\932789160.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(df[i].mode()[0],inplace = True)


In [107]:
from sklearn.preprocessing import MinMaxScaler

In [108]:
col = not_obj(encoded_train)
minmax = MinMaxScaler()
encoded_train[col] = minmax.fit_transform(encoded_train[col])

In [109]:
col = not_obj(encoded_test_X)
minmax = MinMaxScaler()
encoded_test_X[col] = minmax.fit_transform(encoded_test_X[col])

In [110]:
from sklearn.preprocessing import LabelEncoder

In [111]:
def obj_to_int(df,col):
    for i in col:
        label = LabelEncoder()
        df[i] = label.fit_transform(df[i])

In [112]:
col = obj(encoded_train)
obj_to_int(encoded_train,col)

In [113]:
col = obj(encoded_test_X)
obj_to_int(encoded_test_X,col)

In [114]:
encoded_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   int32  
 1   Destination   8693 non-null   int32  
 2   Age           8693 non-null   float64
 3   RoomService   8693 non-null   float64
 4   FoodCourt     8693 non-null   float64
 5   ShoppingMall  8693 non-null   float64
 6   Spa           8693 non-null   float64
 7   VRDeck        8693 non-null   float64
 8   Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), int32(2)
memory usage: 484.0 KB


In [115]:
encoded_test_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   int32  
 1   Destination   4277 non-null   int32  
 2   Age           4277 non-null   float64
 3   RoomService   4277 non-null   float64
 4   FoodCourt     4277 non-null   float64
 5   ShoppingMall  4277 non-null   float64
 6   Spa           4277 non-null   float64
 7   VRDeck        4277 non-null   float64
dtypes: float64(6), int32(2)
memory usage: 234.0 KB


In [116]:
encoded_test_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Transported  4277 non-null   bool 
dtypes: bool(1)
memory usage: 4.3 KB


# Splitting

In [117]:
train_X = encoded_train.drop('Transported',axis=1)
train_y = encoded_train['Transported']

# Training

In [128]:
from sklearn.svm import SVC

In [129]:
svc_model = SVC()
svc_model.fit(train_X,train_y)

SVC()

In [130]:
pred = svc_model.predict(encoded_test_X)

In [131]:
from sklearn.metrics import accuracy_score

In [132]:
acc = accuracy_score(encoded_test_y,pred)
acc

0.5333177460837035

In [134]:
test_y['Transported'] = pred
test_y.to_csv('Prediction_home.csv',index = False)

In [135]:
from sklearn.ensemble import RandomForestClassifier

In [136]:
rfc_model = RandomForestClassifier()
rfc_model.fit(train_X,train_y)

RandomForestClassifier()

In [137]:
pred = rfc_model.predict(encoded_test_X)

In [138]:
acc = accuracy_score(encoded_test_y,pred)

In [139]:
acc

0.5150806640168343

In [140]:
test_y['Transported'] = pred
test_y.to_csv('Prediction_home.csv',index = False)